In [24]:
#Bibliotecas Utilizadas
import IPython as IP
import CoolProp
import CoolProp.CoolProp as CP #Propriedades termodinâmicas
import CoolProp.Plots as CPP #Plotando gráficos
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

### Gráfico auxiliar mostrando diagrama Ph 
![](Pressure-enthalpy-diagram-for-a-standard-vapor-compression-cycle-with-four-thermodynamic.png)


In [25]:
# Importando dados do compressor
compressor_data = pd.read_csv('Dados_Compressor.csv',sep=';',index_col='ID',decimal=',')

# Vendo as primeiras linhas
compressor_data.head()

,Rot,Condicao,T_evap,T_cond,T_sup,T_sub,T_box,T_suc,T_corpo,m,W,delta_h_evap,CAP,COP,rho,m_ideal,eta_v,delta_h_comp,W_ise,eta_g
ID,,,,,,,,,,,,,,,,,,,,
1,3000,A,5.0,45.0,10.0,35.0,35,10.0,68.42,84.30,963.82,171700,4020.49,4.17,34.71,93.73,0.90,30.06,703.88,0.73
2,3000,B,5.0,60.0,10.0,35.0,35,10.0,85.23,79.40,1312.87,172540,3805.38,2.90,34.71,93.73,0.85,39.97,881.54,0.67
3,3000,C,15.0,45.0,20.0,35.0,35,20.0,62.96,117.76,944.62,174420,5705.38,6.04,47.09,127.13,0.93,21.20,693.46,0.73
4,3000,D,15.0,60.0,20.0,35.0,35,20.0,86.35,111.46,1357.54,175260,5426.39,4.00,47.09,127.13,0.88,30.73,951.46,0.70
5,3000,E,10.0,45.0,15.0,35.0,35,15.0,68.22,100.14,948.79,173250,4819.27,5.08,40.33,108.88,0.92,25.56,711.00,0.75


### Significado de cada coluna

`Rot` _Indica a rotação do compressor_ (RPM)

`Condicao` _Indicador de uma condição de super-aquecimento e sub-resfriamento_ (-)

`T_evap` _Temperatura de Evaporação_ (ºC)

`T_cond` _Temperatura de Condensação_ (ºC)

`T_sup` _Temperatura de Super-aquecimento_ (ºC)

`T_sub` _Temperatura de Sub-resfriamento_ (ºC)

`T_box` _Temperatura da (...)_ (ºC)

`T_suc` _Temperatura de (...)_ (ºC)

`T_corpo` _Temperatura do (...)_ (ºC)

`m` _Vazão mássica real_ (Kg/h) 

`W` _Potência Elétrica consumida pelo Compressor - real_ (W)

`delta_h_evap` _Variacão de entalpia no evaporador_ (J/kg)

`CAP` _Capacidade de refrigeração do evaporador_ (W)

`COP` _COP do sistema_ (-)

`rho` _Densidade do fluido refrigerante_ (kg/m3)

`m_ideal` _Vazão mássica ideal_ (Kg/h)

`eta_v` _Rendimento Volumétrico do Compressor_ (-)

`delta_h_comp` _Variação de entalpia no compressor_ (J/kg)

`W_ise` _Potência Elétrica consumida pelo Compressor - ideal_ (W)

`eta_g` _Rendimento Global do Compressor_ (-)


### Montado a tabela com os dados de desempenho do Compressor

In [26]:
# Setando Líquido Refrigerante a ser trabalhado
coolant = 'R410A'

# Criando DataFrame para armazenar resultados relativos a esse líquido refrigenrante
target_data = pd.DataFrame()

# Colunas que permanecem inalteradas
target_data['Rot'] = compressor_data['Rot']
target_data['Condicao'] = compressor_data['Condicao']
target_data['T_evap'] = compressor_data['T_evap']
target_data['T_cond'] = compressor_data['T_cond']
target_data['T_sup'] = compressor_data['T_sup']
target_data['T_sub'] = compressor_data['T_sub']
target_data['CAP'] = compressor_data['CAP']
target_data['W'] = compressor_data['W'] # Trabalho Real do compressor
target_data['V_cil'] = 15e-6 # Acrescentando o volume do cilindro do compressor

In [27]:
target_data.head()

,Rot,Condicao,T_evap,T_cond,T_sup,T_sub,CAP,W,V_cil
ID,,,,,,,,,
1,3000,A,5.0,45.0,10.0,35.0,4020.49,963.82,0.000015
2,3000,B,5.0,60.0,10.0,35.0,3805.38,1312.87,0.000015
3,3000,C,15.0,45.0,20.0,35.0,5705.38,944.62,0.000015
4,3000,D,15.0,60.0,20.0,35.0,5426.39,1357.54,0.000015
5,3000,E,10.0,45.0,15.0,35.0,4819.27,948.79,0.000015


In [19]:
# Colunas calculadas a partir das colunas iniciais
target_data['P_evap'] = [CP.PropsSI('P','Q',1,'T',T+273.15,coolant) 
                         for T in target_data['T_evap']] # Pressão de Evaporação
target_data['P_cond'] = [CP.PropsSI('P','Q',1,'T',T+273.15,coolant) 
                         for T in target_data['T_cond']] # Pressão de Condensação
target_data['RP'] = target_data['P_cond']/target_data['P_evap'] # Razão entre as pressões

target_data['h_1'] = [CP.PropsSI('H','T',target_data['T_sup'].iloc[index]+273.15,'P',
                                 target_data['P_evap'].iloc[index],coolant) for index 
                                  in list(target_data.index - 1)] # Entalpia no ponto 1
target_data['h_3'] = [CP.PropsSI('H','T',target_data['T_sub'].iloc[index]+273.15,'P',
                                 target_data['P_cond'].iloc[index],coolant) for index
                                  in list(target_data.index - 1)] # Entalpia no ponto 3

target_data['m'] = [target_data['CAP'].iloc[index]/(target_data['h_1'].iloc[index] - target_data['h_3'].iloc[index]) 
                    for index in list(target_data.index - 1)] # Vazão mássica real

target_data['v_1'] = [1/(CP.PropsSI('D','T',target_data['T_sup'].iloc[i]+273.15,'P',target_data['P_evap'].iloc[i],coolant)) 
                      for i in list(target_data.index - 1)] # Volume Específico no ponto 1

target_data['eta_v'] = (target_data['m']*target_data['v_1']*60)/(target_data['Rot']*target_data['V_cil']) # Efic. Vol.

target_data['s_1'] = [CP.PropsSI('S','T',target_data['T_sup'].iloc[i]+273.15,
                                'P',target_data['P_evap'].iloc[i],coolant)
                                for i in list(target_data.index - 1)] # Entropia no ponto 1

target_data['h_2s'] = [CP.PropsSI('H','S',target_data['s_1'].iloc[index],'P',
                                 target_data['P_cond'].iloc[index],coolant) for index 
                                  in list(target_data.index - 1)] # Entalpia isentrópica no ponto 2

target_data['W_s'] = target_data['m']*(target_data['h_2s'] - target_data['h_1']) # Trabalho Isentrópico

target_data['eta_g'] = target_data['W_s']/target_data['W'] # Eficiência Volumétrica

target_data['Sigma'] = target_data['Rot']/4800 # Razão entre as rotações

### Overview da Tabela

In [20]:
# Descomentar esse trecho para criar um data report
## Importando biblioteca
#from pandas_profiling import ProfileReport
#profile = ProfileReport(target_data, title='Compressor Modeling',html={'style':{'full_width':True}})
#
## Mostrando report dos dados
#profile

# Overview simples
target_data.head(10)

,Rot,Condicao,T_evap,T_cond,T_sup,T_sub,CAP,W,V_cil,P_evap,...,h_1,h_3,m,v_1,eta_v,s_1,h_2s,W_s,eta_g,Sigma
ID,,,,,,,,,,,,,,,,,,,,,
1,3000,A,5.0,45.0,10.0,35.0,4020.49,963.82,0.000015,9.331762e+05,...,428538.488837,256662.500668,0.023392,0.028925,0.902131,1821.361804,458636.510922,704.047134,0.730476,0.625
2,3000,B,5.0,60.0,10.0,35.0,3805.38,1312.87,0.000015,9.331762e+05,...,428538.488837,255826.800678,0.022033,0.028925,0.849732,1821.361804,468570.248650,882.025182,0.671830,0.625
3,3000,C,15.0,45.0,20.0,35.0,5705.38,944.62,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.032669,0.021332,0.929205,1803.362680,452557.513419,694.262622,0.734965,0.625
4,3000,D,15.0,60.0,20.0,35.0,5426.39,1357.54,0.000015,1.254290e+06,...,431305.922776,255826.800678,0.030923,0.021332,0.879559,1803.362680,462116.091576,952.751472,0.701822,0.625
5,3000,E,10.0,45.0,15.0,35.0,4819.27,948.79,0.000015,1.084818e+06,...,430015.304860,256662.500668,0.027800,0.024799,0.919246,1812.324407,455574.246240,710.547718,0.748899,0.625
6,3000,F,10.0,60.0,15.0,35.0,4566.69,1330.93,0.000015,1.084818e+06,...,430015.304860,255826.800678,0.026217,0.024799,0.866889,1812.324407,465319.851393,925.577267,0.695436,0.625
7,3000,G,5.0,50.0,10.0,35.0,3961.33,1067.31,0.000015,9.331762e+05,...,428538.488837,256386.384387,0.023011,0.028925,0.887430,1821.361804,462014.407581,770.302295,0.721723,0.625
8,3000,H,10.0,50.0,15.0,35.0,4745.37,1071.97,0.000015,1.084818e+06,...,430015.304860,256386.384387,0.027331,0.024799,0.903711,1812.324407,458889.310960,789.141820,0.736160,0.625
9,3000,I,15.0,50.0,20.0,35.0,5662.18,1062.17,0.000015,1.254290e+06,...,431305.922776,256386.384387,0.032370,0.021332,0.920714,1803.362680,455810.126532,793.205915,0.746779,0.625


### Reordenando as linhas da tabela (Ordem crescente de RP)

In [21]:
# Data Sorting
target_data.sort_values(by='RP', axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')

### Plotagem dos Gráficos

In [22]:
# Plotando a Variação da Eficiência Volumétrica
fig_v = go.Figure()

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==3000]['RP'],
                           y=target_data[target_data['Rot']==3000]['eta_v'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 3000'))

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==4000]['RP'],
                           y=target_data[target_data['Rot']==4000]['eta_v'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 4000'))

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==4800]['RP'],
                           y=target_data[target_data['Rot']==4800]['eta_v'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 4800'))

fig_v.update_layout(title='Variação da Eficiência Volumétrica do Compressor com RP',
                   xaxis_title='RP (Relação de Pressões Condesador/Evaporador)',
                   yaxis_title='Eficiência Volumétrica do Compressor',hovermode='x')

fig_v.update_layout(
    hoverlabel=dict( 
        font_size=16))
   
fig_v.show()

In [23]:
# Plotando a Variação da Eficiência Volumétrica
fig_v = go.Figure()

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==3000]['RP'],
                           y=target_data[target_data['Rot']==3000]['eta_g'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 3000'))

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==4000]['RP'],
                           y=target_data[target_data['Rot']==4000]['eta_g'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 4000'))

fig_v.add_trace(go.Scatter(x=target_data[target_data['Rot']==4800]['RP'],
                           y=target_data[target_data['Rot']==4800]['eta_g'],
                           hovertemplate=None,
                           line_shape='spline',name='Rotação 4800'))

fig_v.update_layout(title='Variação da Eficiência Global do Compressor com RP',
                   xaxis_title='RP (Relação de Pressões Condesador/Evaporador)',
                   yaxis_title='Eficiência Global do Compressor',hovermode='x')

fig_v.update_layout(
    hoverlabel=dict( 
        font_size=16))
   
fig_v.show()